In [665]:
import numpy as np
import pandas as pd

hydration_after_p = 0.8
hydration_before_p = 0.045
#Wet Weight(g) after pretreatment
#2% 1h, 12% 1h, 2% 3h 12% 3h
w_60c= np.array([7.8302,4.0316,5.45,4.48]) 
w_80c= np.array([6.4508, 5.181, 5.929, 3.9756]) 
w_100c = np.array([5.6054, 4.1634, 5.9361, 3.9318]) 
w_120c = np.array([4.8918, 3.3319, 5.1648, 2.9348]) 

#Dry Weight(g) after pretreatment
dry_w_60c = w_60c * (1-hydration_after_p)
dry_w_80c = w_80c * (1-hydration_after_p)
dry_w_100c = w_100c * (1-hydration_after_p)
dry_w_120c = w_120c * (1-hydration_after_p)

avg_lost_w_60c = dry_w_60c/2
avg_lost_w_80c = dry_w_80c/2
avg_lost_w_100c = dry_w_100c/2
avg_lost_w_120c = dry_w_120c/2

avg_lost_w_60c, avg_lost_w_80c, avg_lost_w_100c, avg_lost_w_120c

(array([0.78302, 0.40316, 0.545  , 0.448  ]),
 array([0.64508, 0.5181 , 0.5929 , 0.39756]),
 array([0.56054, 0.41634, 0.59361, 0.39318]),
 array([0.48918, 0.33319, 0.51648, 0.29348]))

In [666]:
def calculate_dry_weight(w, hydration_after_p):
    return w * (1 - hydration_after_p)

def calculate_hplc(hplc, k, m, dilution_factor=5):
    return np.array([dilution_factor*(value-m)/k for value in hplc])

def calculate_mass(c, volume_H):
    return c * volume_H

def calculate_max(values, dry_masses):
    return [values[i]*value for i, value in enumerate(dry_masses)]

def calculate_yield(m, max_m):
    return np.divide(m, max_m)

def calculate_average(pairs):
    return np.mean(pairs, axis=1)

def reshape_into_pairs(array):
    return array.reshape(-1, 2)

def format(input_array):
    multiplied_array = input_array * 100
    str_array = np.array(multiplied_array, dtype=str)
    extracted_array = np.array([x[:4] for x in str_array])
    return extracted_array

def format2(input_array):
    str_array = np.array(input_array, dtype=str)
    extracted_array = np.array([x[:4] for x in str_array])
    return extracted_array

def compensate_for_hydration(array, dry_masses_after_p ):
    wet_mass_into_hplc = 2 #g
    moisture = 0.8
    dried_sample = wet_mass_into_hplc * (1-moisture)
    multiply_factor_over_pretreatment = dry_masses_after_p/dried_sample
    return array*multiply_factor_over_pretreatment


In [667]:
def preprocess_data(G_samples_hplc, XMG_samples_hplc, glucose_yields, xylmangal_values, volume, dry_masses_after_p, dry_masses_before_p):
    
    pairs_G_C = reshape_into_pairs(G_samples_hplc)
    pairs_XMG_C = reshape_into_pairs(XMG_samples_hplc)

    avgs_G_C = calculate_average(pairs_G_C)
    avgs_XMG_C = calculate_average(pairs_XMG_C)
  
    c_G = calculate_hplc(avgs_G_C, k=315718, m=4623)
    c_XMG = calculate_hplc(avgs_XMG_C, k=317400, m=10216)
    
    m_G_before = calculate_mass(c_G, volume)
    m_XMG_before = calculate_mass(c_XMG, volume)

    print(f"m_G_before: {m_G_before}")
    print(f"m_XMG_before: {m_XMG_before}")
    print(F"dry_masses_after_p: {dry_masses_after_p}")
    #Kompensera för att vi inte tog allt när vi gjorde hplc
    m_G = compensate_for_hydration(m_G_before, dry_masses_after_p)
    m_XMG = compensate_for_hydration(m_XMG_before, dry_masses_after_p)

    
    #Yield
    yield_glucose_procent_process = calculate_yield(m_G, dry_masses_before_p)
    yield_XMG_procent_process = calculate_yield(m_XMG, dry_masses_before_p)
    yield_monosacharides_procent_process = np.add(yield_glucose_procent_process, yield_XMG_procent_process)

    #Selectivity
    selectivity_glucose_procent_process = np.divide(m_G, dry_masses_after_p)
    selectivity_XMG_procent_process = np.divide(m_XMG, dry_masses_after_p)
    selectivity_monosacharices_procent_process = np.divide(m_G+m_XMG, dry_masses_after_p)
    

    results = {
        "Dry W Milled(g)": format2(dry_masses_before_p),
        "Dry W after Pretreatment(g)": format2(dry_masses_after_p),

        "Glucose(g) ": format2(m_G),
        "XMG(g)": format2(m_XMG),
        "Monosacharides(g)": format2(np.add(m_G, m_XMG)),

        "Selctivity Glucose (%)": format(selectivity_glucose_procent_process),
        "Selctivity XMG (%)": format(selectivity_XMG_procent_process),
        "Selctivity Monosacharides (%)": format(selectivity_monosacharices_procent_process),

        "Yield Glucose (%)": format(yield_glucose_procent_process),
        "Yield XMG (%)": format(yield_XMG_procent_process),
        "Yield Monosacharides (%)": format(yield_monosacharides_procent_process),
    }

    return results

In [668]:
"""
#Varmast 1h, varmast 3h, kallast 1h, kallast 3h
#Ex 120C 1h, 120C 3h, 60C 1h, 60C 3h

#CHANGE THESE
milled_masses = np.array([2.0924,2.0960,2.0954,2.0985,2.0923,2.0979,2.0962,2.0943])
Glucose_samples_hplc = np.array([590520.1774,  626867.5736, 693851.4441, 661331.5045, 578989.8003, 675418.0375, 571507.7927, 596580.4333])
XMG_samples_hplc = np.array([149706.8617, 159126.8929,  176542.8571,  168805.7143,  147914.2857,  172857.1429,  146857.1429,  153200.0000])

#Constants
glucose_yields = np.array([39.54, 88.30, 68.42, 65.11, 56.28])/100
xylmangal_values = np.array([25.24, 2.64, 1.97, 9.96, 7.4])/100
volume = 0.02485
hydration_after_p = 0.8

dry_masses_before_p = milled_masses*0.955
average_dry_masses_before_pretreatment = calculate_average(reshape_into_pairs(dry_masses_before_p))
a = average_dry_masses_before_pretreatment

#CHANGE THIS 
dry_masses_after_p = np.array([a[0]*avg_lost_w_120c[0], a[1]*avg_lost_w_120c[2], a[2]*avg_lost_w_60c[0], a[3]*avg_lost_w_60c[2]])
"""

'\n#Varmast 1h, varmast 3h, kallast 1h, kallast 3h\n#Ex 120C 1h, 120C 3h, 60C 1h, 60C 3h\n\n#CHANGE THESE\nmilled_masses = np.array([2.0924,2.0960,2.0954,2.0985,2.0923,2.0979,2.0962,2.0943])\nGlucose_samples_hplc = np.array([590520.1774,  626867.5736, 693851.4441, 661331.5045, 578989.8003, 675418.0375, 571507.7927, 596580.4333])\nXMG_samples_hplc = np.array([149706.8617, 159126.8929,  176542.8571,  168805.7143,  147914.2857,  172857.1429,  146857.1429,  153200.0000])\n\n#Constants\nglucose_yields = np.array([39.54, 88.30, 68.42, 65.11, 56.28])/100\nxylmangal_values = np.array([25.24, 2.64, 1.97, 9.96, 7.4])/100\nvolume = 0.02485\nhydration_after_p = 0.8\n\ndry_masses_before_p = milled_masses*0.955\naverage_dry_masses_before_pretreatment = calculate_average(reshape_into_pairs(dry_masses_before_p))\na = average_dry_masses_before_pretreatment\n\n#CHANGE THIS \ndry_masses_after_p = np.array([a[0]*avg_lost_w_120c[0], a[1]*avg_lost_w_120c[2], a[2]*avg_lost_w_60c[0], a[3]*avg_lost_w_60c[2]])\

In [669]:

"""
data = preprocess_data(Glucose_samples_hplc, XMG_samples_hplc, glucose_yields, xylmangal_values, volume, dry_masses_after_p, dry_masses_before_p)


index = ['120C 1h', '120C 3h', '60C 1h', '60C 3h']
process_data_keys = [
    "Dry W Milled(g)",
    "Dry W after Pretreatment(g)",
    "Glucose(g) ",
    "XMG(g)",
    "Monosacharides(g)",
    "Yield Glucose (%)",
    "Yield XMG (%)",
    "Yield Monosacharides (%)",
    "Selctivity Glucose (%)",
    "Selctivity XMG (%)",
    "Selctivity Monosacharides (%)"
]

# Create dataframes
process_data = {key: data[key] for key in process_data_keys}
process_df = pd.DataFrame(process_data)
process_df.index = index

print('2% NaOH batch - Process Data')
print(f"NREL Yield Glucose: {39.54}")
print(f"NREL Yield XMG: {25.24}")
print(f"NREL Yield Monosacharides: {39.54+25.24}")
print()
process_df
"""


'\ndata = preprocess_data(Glucose_samples_hplc, XMG_samples_hplc, glucose_yields, xylmangal_values, volume, dry_masses_after_p, dry_masses_before_p)\n\n\nindex = [\'120C 1h\', \'120C 3h\', \'60C 1h\', \'60C 3h\']\nprocess_data_keys = [\n    "Dry W Milled(g)",\n    "Dry W after Pretreatment(g)",\n    "Glucose(g) ",\n    "XMG(g)",\n    "Monosacharides(g)",\n    "Yield Glucose (%)",\n    "Yield XMG (%)",\n    "Yield Monosacharides (%)",\n    "Selctivity Glucose (%)",\n    "Selctivity XMG (%)",\n    "Selctivity Monosacharides (%)"\n]\n\n# Create dataframes\nprocess_data = {key: data[key] for key in process_data_keys}\nprocess_df = pd.DataFrame(process_data)\nprocess_df.index = index\n\nprint(\'2% NaOH batch - Process Data\')\nprint(f"NREL Yield Glucose: {39.54}")\nprint(f"NREL Yield XMG: {25.24}")\nprint(f"NREL Yield Monosacharides: {39.54+25.24}")\nprint()\nprocess_df\n'

In [670]:
def create_dataframe(Glucose_samples_hplc, XMG_samples_hplc, indices, dry_masses_after_p,dry_masses_before_p):
    #Constants
    volume = 0.02485
    glucose_yields = np.array([39.54, 88.30, 68.42, 65.11, 56.28])/100,
    xylmangal_values = np.array([25.24, 2.64, 1.97, 9.96, 7.4])/100
    

    data = preprocess_data(Glucose_samples_hplc, XMG_samples_hplc, glucose_yields, xylmangal_values, volume, dry_masses_after_p, dry_masses_before_p)

    process_data_keys = [
        "Dry W Milled(g)",
        "Dry W after Pretreatment(g)",
        "Glucose(g) ",
        "XMG(g)",
        "Monosacharides(g)",
        "Yield Glucose (%)",
        "Yield XMG (%)",
        "Yield Monosacharides (%)",
        "Selctivity Glucose (%)",
        "Selctivity XMG (%)",
        "Selctivity Monosacharides (%)"
    ]

    # Create dataframes
    process_data = {key: data[key] for key in process_data_keys}
    process_df = pd.DataFrame(process_data)
    process_df.index = indices

   

    return process_df


In [671]:
#2% NaOH 60-120%
milled_masses_1 = np.array([2.0924, 2.0960, 2.0954, 2.0985, 2.0923, 2.0979, 2.0962, 2.0943])
dry_masses_before_p_1 = milled_masses_1*0.955
avg_before_p_1 = calculate_average(reshape_into_pairs(dry_masses_before_p_1))
a_1 = avg_before_p_1
dry_masses_after_p_1 = np.array([a_1[0]*avg_lost_w_120c[0], a_1[1]*avg_lost_w_120c[2], a_1[2]*avg_lost_w_60c[0], a_1[3]*avg_lost_w_60c[2]])


#12% NaOH 60-120% 
milled_masses_2 = np.array([2.0924, 2.0960, 2.0954, 2.0985, 2.0923, 2.0979, 2.0962, 2.0943])
dry_masses_before_p_2 = milled_masses_2*0.955
avg_before_p_2 = calculate_average(reshape_into_pairs(dry_masses_before_p_2))
a_2 = avg_before_p_2
dry_masses_after_p_2 = np.array([a_2[0]*avg_lost_w_120c[1], a_2[1]*avg_lost_w_120c[3], a_2[2]*avg_lost_w_60c[1], a_2[3]*avg_lost_w_60c[3]])


#12% NaOH 80-100%
milled_masses_3 = np.array([2.0003, 2.0024, 1.9970, 2.0050, 2.0008, 1.9008, 1.9980, 2.002])
dry_masses_before_p_3 = milled_masses_3*0.955
avg_before_p_3 = calculate_average(reshape_into_pairs(dry_masses_before_p_3))
a_3 = avg_before_p_3
dry_masses_after_p_3 = np.array([a_3[0]*avg_lost_w_100c[1], a_3[1]*avg_lost_w_100c[3], a_3[2]*avg_lost_w_80c[1], a_3[3]*avg_lost_w_80c[3]])


#2% NaOH 80-100%
milled_masses_4 = np.array([2.0040, 1.9977, 1.9997, 2.0030, 2.0003, 1.9984, 2.0020, 1.9966])
dry_masses_before_p_4 = milled_masses_4*0.955
avg_before_p_4 = calculate_average(reshape_into_pairs(dry_masses_before_p_4))
a_4 = avg_before_p_4
dry_masses_after_p_4 = np.array([a_4[0]*avg_lost_w_100c[0], a_4[1]*avg_lost_w_100c[2], a_4[2]*avg_lost_w_80c[0], a_4[3]*avg_lost_w_80c[2]])

#avg_before_p_1, avg_before_p_2, avg_before_p_3, avg_before_p_4
dry_masses_after_p_1, dry_masses_after_p_2, dry_masses_after_p_3, dry_masses_after_p_4

(array([0.97834092, 1.03429626, 1.56668247, 1.09052524]),
 array([0.66636701, 0.58771931, 0.80665079, 0.89643176]),
 array([0.79574617, 0.75134929, 0.96522755, 0.7593396 ]),
 array([1.07108642, 1.13456041, 1.23170237, 1.13204265]))

In [672]:
studies = [
    {
        "Glucose_samples_hplc": np.array([590520.1774, 626867.5736, 693851.4441, 661331.5045, 578989.8003, 675418.0375, 571507.7927, 596580.4333]),
        "XMG_samples_hplc": np.array([149706.8617, 159126.8929, 176542.8571, 168805.7143, 147914.2857, 172857.1429, 146857.1429, 153200.0000]),
        "dry_masses_after_p": dry_masses_after_p_1,
        "dry_masses_before_p": avg_before_p_1,
        "indices": ['120C 1h', '120C 3h', '60C 1h', '60C 3h']
     },
    {
        "Glucose_samples_hplc": np.array([1164687.9925, 1166254.1841, 1219303.4481, 1217971.6083, 772940.5642, 800664.2413, 816496.2310, 777523.5907]),
        "XMG_samples_hplc": np.array([0, 0, 0, 0, 103180.0916, 106556.1953, 79071.6490, 74139.5075]),
        "dry_masses_after_p": dry_masses_after_p_2,
        "dry_masses_before_p": avg_before_p_2,
        "indices": ['120C 1h', '120C 3h', '60C 1h', '60C 3h']
    },
    {
        "Glucose_samples_hplc": np.array([1084281.8242, 1437375.2409, 1095570.4941, 1269949.5440, 0,0, 1125741.7195, 1193543.1789]),
        "XMG_samples_hplc": np.array([0, 97365.4265, 0, 0, 0, 0, 0, 0]),
        "dry_masses_after_p": dry_masses_after_p_3,
        "dry_masses_before_p": avg_before_p_3,
        "indices": ['100C 1h', '100C 3h', '80C 1h', '80C 3h']

    },
    {
        "Glucose_samples_hplc": np.array([804353.4156, 739282.6040, 745575.4034, 745801.7998, 727440.5672, 924715.2307, 880340.6993, 899655.9408]),
        "XMG_samples_hplc": np.array([0, 154173.2942, 195054.4891, 202082.0536, 159049.2005, 205812.6120, 189111.6415, 190132.9875]),
        "dry_masses_after_p": dry_masses_after_p_4,
        "dry_masses_before_p": avg_before_p_4,
        "indices": ['100C 1h', '100C 3h', '80C 1h', '80C 3h']
    }
]

# For each study, create a dataframe and store it in a list
dataframes = []


print()
print("Data for studies of various pretreatment conditions:")
print()
print(f"NREL Data:")
print(f"NREL Yield Glucose: {39.54}")
print(f"NREL Yield XMG: {25.24}")
print(f"NREL Yield Monosacharides: {39.54+25.24}")
print()
labels = [['2% 60-120'], ['12% 60-120'], ['12% 80-100'], ['2% 80-100']]
for i, study in enumerate(studies):
    df = create_dataframe(**study)
    styled_df = df.style.background_gradient(cmap='viridis')
    dataframes.append(styled_df)
    print(f'Data for study: {labels[i][0]}:')
    display(styled_df)




Data for studies of various pretreatment conditions:

NREL Data:
NREL Yield Glucose: 39.54
NREL Yield XMG: 25.24
NREL Yield Monosacharides: 64.78

dry_masses_after_p: [0.97834092 1.03429626 1.56668247 1.09052524]
Data for study: 2% 60-120:


,Dry W Milled(g),Dry W after Pretreatment(g),Glucose(g),XMG(g),Monosacharides(g),Yield Glucose (%),Yield XMG (%),Yield Monosacharides (%),Selctivity Glucose (%),Selctivity XMG (%),Selctivity Monosacharides (%)
120C 1h,1.99,0.97,0.58,0.13,0.71,29.0,6.90,35.9,59.4,14.1,73.5
120C 3h,2.00,1.03,0.68,0.16,0.84,34.1,8.21,42.4,66.2,15.8,82.1
60C 1h,2.00,1.56,0.95,0.23,1.18,47.9,11.5,59.4,61.2,14.6,75.9
60C 3h,2.00,1.09,0.62,0.14,0.77,31.0,7.45,38.5,57.0,13.6,70.6


dry_masses_after_p: [0.66636701 0.58771931 0.80665079 0.89643176]
Data for study: 12% 60-120:


,Dry W Milled(g),Dry W after Pretreatment(g),Glucose(g),XMG(g),Monosacharides(g),Yield Glucose (%),Yield XMG (%),Yield Monosacharides (%),Selctivity Glucose (%),Selctivity XMG (%),Selctivity Monosacharides (%)
120C 1h,1.99,0.66,0.76,-0.0,0.75,38.0,-0.3,37.7,114.,-0.9,113.
120C 3h,2.00,0.58,0.70,-0.0,0.69,35.0,-0.2,34.7,119.,-0.9,118.
60C 1h,2.00,0.80,0.62,0.07,0.69,31.0,3.73,34.7,76.9,9.26,86.2
60C 3h,2.00,0.89,0.69,0.05,0.75,34.9,2.91,37.8,77.9,6.49,84.4


dry_masses_after_p: [0.79574617 0.75134929 0.96522755 0.7593396 ]
Data for study: 12% 80-100:


,Dry W Milled(g),Dry W after Pretreatment(g),Glucose(g),XMG(g),Monosacharides(g),Yield Glucose (%),Yield XMG (%),Yield Monosacharides (%),Selctivity Glucose (%),Selctivity XMG (%),Selctivity Monosacharides (%)
100C 1h,1.91,0.79,0.98,0.02,1.01,51.4,1.56,53.0,123.,3.76,127.
100C 3h,1.91,0.75,0.87,-0.0,0.86,45.5,-0.3,45.1,115.,-0.9,114.
80C 1h,1.86,0.96,-0.0,-0.0,-0.0,-0.2,-0.5,-0.7,-0.4,-0.9,-1.4
80C 3h,1.90,0.75,0.86,-0.0,0.85,45.1,-0.3,44.7,113.,-0.9,112.


dry_masses_after_p: [1.07108642 1.13456041 1.23170237 1.13204265]
Data for study: 2% 80-100:


,Dry W Milled(g),Dry W after Pretreatment(g),Glucose(g),XMG(g),Monosacharides(g),Yield Glucose (%),Yield XMG (%),Yield Monosacharides (%),Selctivity Glucose (%),Selctivity XMG (%),Selctivity Monosacharides (%)
100C 1h,1.91,1.07,0.80,0.07,0.87,42.3,3.66,45.9,75.4,6.54,82.0
100C 3h,1.91,1.13,0.82,0.20,1.03,43.2,10.9,54.2,72.9,18.4,91.3
80C 1h,1.90,1.23,0.99,0.20,1.20,52.1,10.8,63.0,80.8,16.8,97.6
80C 3h,1.90,1.13,0.98,0.19,1.18,51.6,10.4,62.0,87.1,17.5,104.
